<a href="https://colab.research.google.com/github/h5ng/GNN/blob/master/gpt_gnn_preprocess_OAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

In [3]:
!pip install texttable

In [4]:
import sys
from google.colab import drive
drive.mount('/content/dirve', force_remount=True)

gpt_gnn_path = '/content/dirve/MyDrive/GPT-GNN-master/example_OAG/GPT_GNN'
sys.path.append(gpt_gnn_path)

Mounted at /content/dirve


In [5]:
from transformers import *

from data import *
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm

In [6]:
from argparse import Namespace

args = {
    "input_dir": gpt_gnn_path + '/data',
    "output_dir": "./data/oag_output",
    "cuda": 0,
    "domain": "_CS",
    "citation_bar": 1
}

args = Namespace(**args)

test_time_bar = 2016



In [7]:
# 각 논문의 cite 수
cite_dict = defaultdict(lambda: 0)
with open(args.input_dir + '/PR%s_20190919.tsv' % args.domain, encoding="UTF8") as fin:
  fin.readline()
  for l in tqdm(fin, total = sum(1 for line in open(args.input_dir + '/PR%s_20190919.tsv' % args.domain, encoding="UTF8"))):
    l = l[:-1].split('\t')
    cite_dict[l[1]] += 1

100%|█████████▉| 31441551/31441552 [00:25<00:00, 1252494.84it/s]


In [8]:
pfl = defaultdict(lambda: {})
with open(args.input_dir + '/Papers%s_20190919.tsv' % args.domain, encoding="UTF8") as fin:
  fin.readline()
  for l in tqdm(fin, total = sum(1 for line in open(args.input_dir + '/Papers%s_20190919.tsv' % args.domain, encoding="UTF8"))):
    l = l[:-1].split('\t')
    bound = min(2020 - int(l[1]), 20) * args.citation_bar
    if cite_dict[l[0]] < bound or l[0] == '' or l[1] == '' or l[2] == '' or l[3] == '' and l[4] == '' or int(l[1]) < 1900:
      continue
    pi = {'id': l[0], 'title': l[2], 'type': 'paper', 'time': int(l[1])}
    pfl[l[0]] = pi

100%|█████████▉| 5597605/5597606 [00:15<00:00, 356097.48it/s]


In [9]:
if args.cuda != -1:
  device = torch.device("cuda:" + str(args.cuda))
else:
  device = torch.device("cpu")

In [10]:
!pip install sentencepiece

In [11]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased',
                                   output_hidden_states=True,
                                   output_attentions=True).to(device)

In [12]:
# abstract of paper
with open(args.input_dir + '/PAb%s_20190919.tsv' % args.domain, encoding="UTF8") as fin:
  fin.readline()
  for l in tqdm(fin, total = sum(1 for line in open(args.input_dir + '/PAb%s_20190919.tsv' % args.domain, 'r', encoding="UTF8"))):
    try:
      l = l.split('\t')
      if l[0] in pfl:
        input_ids = torch.tensor([tokenizer.encode(pfl[l[0]]['title'])]).to(device)[:, :64]
        if len(input_ids[0]) < 4:
          continue
        all_hidden_states, all_attentions = model(input_ids)[-2:]
        rep = (all_hidden_states[-2][0] * all_attentions[-2][0].mean(dim=0).mean(dim=0).view(-1, 1)).sum(dim=0)
        pfl[l[0]]['emb'] = rep.tolist()
    except Exception as e:
      print(e)
      break

 18%|█▊        | 806883/4542602 [27:15<1:49:08, 570.44it/s]

KeyboardInterrupt: ignored